# Dzień 1 - Moduł 3: Podstawowe operacje w NLP

## Cele modułu:
- Tokenizacja tekstu na różne sposoby
- Lematyzacja i stemming
- Usuwanie stop words i normalizacja
- POS tagging i dependency parsing
- Praktyczne przetwarzanie struktur językowych

In [ ]:
# Import potrzebnych bibliotek
import nltk
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
import re

# Pobierz niezbędne zasoby
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("✅ Biblioteki załadowane!")

## 3.1 Tokenizacja

**Tokenizacja** to proces dzielenia tekstu na mniejsze jednostki (tokeny).

### Rodzaje tokenizacji:
- **Tokenizacja zdań** - podział na zdania
- **Tokenizacja słów** - podział na słowa
- **Tokenizacja podwyrazów** - podział na części słów (BPE, WordPiece)
- **Tokenizacja znaków** - podział na pojedyncze znaki

### Po co tokenizacja?
- Przygotowanie danych do analizy
- Podstawa dla wszystkich operacji NLP
- Różne modele wymagają różnej tokenizacji

In [ ]:
# Przykład tekstu
tekst = """
Dr. Jan Kowalski pracuje w firmie AI-Tech sp. z o.o. Od 2020 r. zajmuje się 
przetwarzaniem języka naturalnego (NLP). Jego e-mail to: jan.kowalski@ai-tech.pl. 
"To fascynująca dziedzina!" - powiedział w wywiadzie.
"""

print("=== TEKST ORYGINALNY ===")
print(tekst)

In [ ]:
# Tokenizacja zdań - NLTK
print("\n=== TOKENIZACJA ZDAŃ (NLTK) ===")
zdania_nltk = sent_tokenize(tekst, language='polish')

for i, zdanie in enumerate(zdania_nltk, 1):
    print(f"{i}. {zdanie.strip()}")

In [ ]:
# Tokenizacja słów - NLTK
print("\n=== TOKENIZACJA SŁÓW (NLTK) ===")
slowa_nltk = word_tokenize(tekst, language='polish')
print(f"Liczba tokenów: {len(slowa_nltk)}")
print(f"Pierwsze 20 tokenów: {slowa_nltk[:20]}")

In [ ]:
# Tokenizacja - spaCy (bardziej zaawansowana)
try:
    nlp = spacy.load("pl_core_news_sm")
    print("\n=== TOKENIZACJA (spaCy - model polski) ===")
except:
    nlp = spacy.load("en_core_web_sm")
    print("\n=== TOKENIZACJA (spaCy - model angielski) ===")
    tekst = "Dr. Smith works at AI-Tech Inc. since 2020. He specializes in NLP!"

doc = nlp(tekst)

print("\nZdania:")
for i, sent in enumerate(doc.sents, 1):
    print(f"{i}. {sent.text.strip()}")

print(f"\nTokeny: {[token.text for token in doc][:20]}")

In [ ]:
# Porównanie różnych metod tokenizacji
tekst_prosty = "Nie lubię tego. To jest złe!!!"

print("=== PORÓWNANIE METOD TOKENIZACJI ===")
print(f"Tekst: {tekst_prosty}\n")

# Metoda 1: Proste split()
print("1. split():", tekst_prosty.split())

# Metoda 2: Regex
print("2. Regex:", re.findall(r'\w+|[^\w\s]', tekst_prosty))

# Metoda 3: NLTK
print("3. NLTK:", word_tokenize(tekst_prosty, language='polish'))

# Metoda 4: spaCy
print("4. spaCy:", [token.text for token in nlp(tekst_prosty)])

## 3.2 Lematyzacja i Stemming

### Stemming
- Odcina końcówki słów algorytmicznie
- Szybkie, ale mniej dokładne
- Może tworzyć "nie-słowa"
- Przykład: "biegający" → "bieg"

### Lematyzacja
- Sprowadza słowa do formy podstawowej (lematu)
- Używa słownika i kontekstu
- Wolniejsze, ale dokładniejsze
- Przykład: "biegający" → "biegać"

### Kiedy używać?
- **Stemming**: wyszukiwanie, clustering
- **Lematyzacja**: analiza semantyczna, klasyfikacja

In [ ]:
# Stemming - Porter Stemmer (dla angielskiego)
porter = PorterStemmer()

slowa_en = ["running", "runs", "ran", "runner", "easily", "fairly"]

print("=== STEMMING (Porter - angielski) ===")
for slowo in slowa_en:
    stem = porter.stem(slowo)
    print(f"{slowo:15} -> {stem}")

In [ ]:
# Stemming - Snowball Stemmer (wielojęzyczny)
stemmer_pl = SnowballStemmer('polish')

slowa_pl = ["biegający", "biegała", "biegać", "biegnie", "programowanie", "programować", "programista"]

print("\n=== STEMMING (Snowball - polski) ===")
for slowo in slowa_pl:
    stem = stemmer_pl.stem(slowo)
    print(f"{slowo:20} -> {stem}")

In [ ]:
# Lematyzacja - NLTK WordNet (angielski)
lemmatizer = WordNetLemmatizer()

print("\n=== LEMATYZACJA (WordNet - angielski) ===")
for slowo in slowa_en:
    lemma = lemmatizer.lemmatize(slowo, pos='v')  # pos='v' oznacza czasownik
    print(f"{slowo:15} -> {lemma}")

In [ ]:
# Lematyzacja - spaCy (lepsze dla wielu języków)
try:
    nlp_pl = spacy.load("pl_core_news_sm")
    tekst_pl = "Programiści programują programy w różnych językach programowania."
    doc = nlp_pl(tekst_pl)
    jezyk = "polski"
except:
    nlp_pl = spacy.load("en_core_web_sm")
    tekst_pl = "The runners are running in different running competitions."
    doc = nlp_pl(tekst_pl)
    jezyk = "angielski"

print(f"\n=== LEMATYZACJA (spaCy - {jezyk}) ===")
print(f"Tekst: {tekst_pl}\n")

for token in doc:
    if not token.is_punct and not token.is_space:
        print(f"{token.text:20} -> {token.lemma_}")

In [ ]:
# Porównanie: Stemming vs Lematyzacja
tekst_test = "The striped bats are hanging on their feet for best"
tokens = word_tokenize(tekst_test)

print("\n=== PORÓWNANIE: STEMMING vs LEMATYZACJA ===")
print(f"{'Oryginał':<20} {'Stemming':<20} {'Lematyzacja':<20}")
print("-" * 60)

for token in tokens:
    if token.isalpha():  # tylko litery
        stem = porter.stem(token)
        lemma = lemmatizer.lemmatize(token)
        print(f"{token:<20} {stem:<20} {lemma:<20}")

## 3.3 Usuwanie Stop Words i Normalizacja

### Stop Words
Najczęstsze słowa w języku, które zazwyczaj niosą mało znaczenia:
- Polski: "i", "w", "na", "z", "się", "jest", "to"
- Angielski: "the", "is", "at", "which", "on", "a", "an"

### Kiedy usuwać stop words?
- ✅ Klasyfikacja tekstu
- ✅ Clustering dokumentów
- ✅ Analiza tematów
- ❌ Tłumaczenia maszynowe
- ❌ Analiza sentymentu ("not good" vs "good")
- ❌ Rozpoznawanie nazwanych encji

### Normalizacja
- Lowercase (małe litery)
- Usuwanie znaków specjalnych
- Usuwanie liczb
- Usuwanie białych znaków

In [ ]:
# Stop words - NLTK
print("=== STOP WORDS ===")

stop_words_pl = set(stopwords.words('polish'))
stop_words_en = set(stopwords.words('english'))

print(f"Język polski - liczba stop words: {len(stop_words_pl)}")
print(f"Przykłady PL: {list(stop_words_pl)[:20]}")

print(f"\nJęzyk angielski - liczba stop words: {len(stop_words_en)}")
print(f"Przykłady EN: {list(stop_words_en)[:20]}")

In [ ]:
# Usuwanie stop words - przykład
tekst = "To jest przykładowy tekst w języku polskim. Pokazuje jak usuwać stop words."

print("\n=== USUWANIE STOP WORDS ===")
print(f"Oryginalny tekst:\n{tekst}\n")

# Tokenizacja
tokens = word_tokenize(tekst.lower(), language='polish')
print(f"Tokeny: {tokens}\n")

# Usunięcie stop words
filtered_tokens = [token for token in tokens if token not in stop_words_pl and token.isalpha()]
print(f"Po usunięciu stop words: {filtered_tokens}")

# Złączenie z powrotem
tekst_bez_stopwords = ' '.join(filtered_tokens)
print(f"\nTekst bez stop words:\n{tekst_bez_stopwords}")

In [ ]:
# Pełna normalizacja tekstu - pipeline
def normalize_text(text, remove_stopwords=True, lemmatize=True, language='polish'):
    """
    Kompleksowa normalizacja tekstu.
    
    Args:
        text: Tekst do normalizacji
        remove_stopwords: Czy usunąć stop words
        lemmatize: Czy lematyzować
        language: Język tekstu
    
    Returns:
        Znormalizowany tekst
    """
    # 1. Lowercase
    text = text.lower()
    
    # 2. Usuń znaki specjalne i liczby
    text = re.sub(r'[^a-ząćęłńóśźż\s]', '', text)
    
    # 3. Tokenizacja
    tokens = word_tokenize(text, language=language)
    
    # 4. Usuń stop words
    if remove_stopwords:
        stop_words = set(stopwords.words(language))
        tokens = [t for t in tokens if t not in stop_words]
    
    # 5. Lematyzacja (używamy spaCy)
    if lemmatize:
        try:
            nlp = spacy.load("pl_core_news_sm" if language == 'polish' else "en_core_web_sm")
            doc = nlp(' '.join(tokens))
            tokens = [token.lemma_ for token in doc if not token.is_space]
        except:
            pass  # Jeśli nie ma modelu, pomiń lematyzację
    
    # 6. Usuń puste tokeny
    tokens = [t for t in tokens if t.strip()]
    
    return ' '.join(tokens)

# Test
tekst_test = "Dr. Jan Kowalski (ur. 1980) pracuje w AI-Tech!!! Zajmuje się NLP-em od 2020 roku."

print("\n=== PEŁNA NORMALIZACJA ===")
print(f"Oryginał:\n{tekst_test}\n")
print(f"Znormalizowany:\n{normalize_text(tekst_test)}")

## 3.4 POS Tagging (Part-of-Speech Tagging)

**POS Tagging** to oznaczanie części mowy dla każdego słowa:
- Rzeczownik (noun)
- Czasownik (verb)
- Przymiotnik (adjective)
- Przysłówek (adverb)
- itp.

### Zastosowania:
- Analiza składniowa
- Ekstrakcja informacji
- Poprawianie lematyzacji
- Rozumienie struktury zdania

In [ ]:
# POS Tagging - NLTK (angielski)
from nltk import pos_tag

tekst_en = "The quick brown fox jumps over the lazy dog."
tokens_en = word_tokenize(tekst_en)
pos_tags = pos_tag(tokens_en)

print("=== POS TAGGING (NLTK - angielski) ===")
print(f"Tekst: {tekst_en}\n")

for word, tag in pos_tags:
    print(f"{word:15} -> {tag:10}")

print("\nLegenda:")
print("DT = Determiner, JJ = Adjective, NN = Noun, VBZ = Verb, IN = Preposition")

In [ ]:
# POS Tagging - spaCy (lepsze, wielojęzyczne)
try:
    nlp = spacy.load("pl_core_news_sm")
    tekst = "Szybki brązowy lis przeskoczył przez leniwego psa."
    jezyk = "polski"
except:
    nlp = spacy.load("en_core_web_sm")
    tekst = "The quick brown fox jumps over the lazy dog."
    jezyk = "angielski"

doc = nlp(tekst)

print(f"\n=== POS TAGGING (spaCy - {jezyk}) ===")
print(f"Tekst: {tekst}\n")
print(f"{'Token':<20} {'POS':<10} {'Tag':<10} {'Opis'}")
print("-" * 70)

for token in doc:
    if not token.is_space:
        print(f"{token.text:<20} {token.pos_:<10} {token.tag_:<10} {spacy.explain(token.pos_)}")

In [ ]:
# Ekstrakcja rzeczowników i czasowników
print("\n=== EKSTRAKCJA CZĘŚCI MOWY ===")

rzeczowniki = [token.text for token in doc if token.pos_ == "NOUN"]
czasowniki = [token.text for token in doc if token.pos_ == "VERB"]
przymiotniki = [token.text for token in doc if token.pos_ == "ADJ"]

print(f"Rzeczowniki: {rzeczowniki}")
print(f"Czasowniki: {czasowniki}")
print(f"Przymiotniki: {przymiotniki}")

## 3.5 Dependency Parsing

**Dependency Parsing** to analiza zależności składniowych między słowami.

### Co pokazuje?
- Relacje między słowami
- Struktura gramatyczna zdania
- Podmiot, orzeczenie, dopełnienie
- Modyfikatory

### Zastosowania:
- Ekstrakcja relacji
- Rozumienie pytań
- Analiza sentymentu zorientowana na aspekty
- Generowanie odpowiedzi

In [ ]:
# Dependency Parsing - spaCy
try:
    nlp = spacy.load("pl_core_news_sm")
    tekst = "Jan dał Marii książkę o sztucznej inteligencji."
except:
    nlp = spacy.load("en_core_web_sm")
    tekst = "John gave Mary a book about artificial intelligence."

doc = nlp(tekst)

print("=== DEPENDENCY PARSING ===")
print(f"Tekst: {tekst}\n")
print(f"{'Token':<20} {'Dep':<15} {'Head':<20} {'Opis'}")
print("-" * 80)

for token in doc:
    print(f"{token.text:<20} {token.dep_:<15} {token.head.text:<20} {spacy.explain(token.dep_)}")

In [ ]:
# Wizualizacja drzewa zależności (w notebooku)
from spacy import displacy

print("\n=== WIZUALIZACJA DRZEWA ZALEŻNOŚCI ===")

# Renderowanie w notebooku
displacy.render(doc, style="dep", jupyter=True, options={'distance': 120})

In [ ]:
# Ekstrakcja podmiotu i dopełnienia
print("\n=== EKSTRAKCJA PODMIOTÓW I DOPEŁNIEŃ ===")

for token in doc:
    if "subj" in token.dep_:  # podmiot
        print(f"Podmiot: {token.text}")
    if "obj" in token.dep_:  # dopełnienie
        print(f"Dopełnienie: {token.text}")
    if token.pos_ == "VERB":
        print(f"Czasownik: {token.text}")

## Ćwiczenie kompleksowe

**Zadanie**: Stwórz pipeline do przetwarzania tekstu, który:
1. Normalizuje tekst
2. Usuwa stop words
3. Lematyzuje słowa
4. Wyciąga rzeczowniki i czasowniki
5. Znajduje nazwane encje

In [ ]:
def advanced_text_processing(text, language='polish'):
    """
    Kompleksowe przetwarzanie tekstu.
    """
    # Załaduj model spaCy
    try:
        if language == 'polish':
            nlp = spacy.load("pl_core_news_sm")
        else:
            nlp = spacy.load("en_core_web_sm")
    except:
        print("⚠️ Model nie jest zainstalowany")
        return None
    
    # Przetwórz tekst
    doc = nlp(text)
    
    # 1. Podstawowe informacje
    print("=== ANALIZA TEKSTU ===")
    print(f"Tekst: {text}\n")
    print(f"Liczba tokenów: {len(doc)}")
    print(f"Liczba zdań: {len(list(doc.sents))}\n")
    
    # 2. Lematyzacja
    print("=== LEMATY ===")
    lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    print(lemmas)
    print()
    
    # 3. Części mowy
    print("=== CZĘŚCI MOWY ===")
    rzeczowniki = [token.text for token in doc if token.pos_ == "NOUN"]
    czasowniki = [token.text for token in doc if token.pos_ == "VERB"]
    print(f"Rzeczowniki: {rzeczowniki}")
    print(f"Czasowniki: {czasowniki}\n")
    
    # 4. Nazwane encje
    print("=== NAZWANE ENCJE ===")
    if doc.ents:
        for ent in doc.ents:
            print(f"{ent.text:<30} -> {ent.label_:<15}")
    else:
        print("Nie znaleziono nazwanych encji.")
    
    return doc

# Test
tekst_test = """
Adam Mickiewicz napisał Pana Tadeusza w Paryżu. 
To arcydzieło polskiej literatury romantycznej ukazało się w 1834 roku.
"""

doc = advanced_text_processing(tekst_test, language='polish')

## Podsumowanie Modułu 3

✅ Nauczyliśmy się tokenizacji na różne sposoby

✅ Zrozumieliśmy różnicę między stemmingiem a lematyzacją

✅ Poznaliśmy techniki normalizacji i usuwania stop words

✅ Opanowaliśmy POS tagging i dependency parsing

✅ Stworzyliśmy kompleksowy pipeline przetwarzania tekstu

---

**Następny krok**: Warsztaty praktyczne - Zastosowanie poznanych technik w praktyce!